In [2]:
import nltk
from collections import defaultdict, Counter
import gensim, logging
import nltk
from gensim.models.keyedvectors import KeyedVectors
from gensim.similarities.index import AnnoyIndexer
from multiprocessing import Process, Pool
import os
import collections
from random import shuffle
from copy import deepcopy
from collections import OrderedDict
import pickle
import networkx as nx
import matplotlib.pyplot as plt
from operator import itemgetter
from scipy.stats import spearmanr
import pandas as pd
import numpy as np
from gensim.models.wrappers import FastText

#### Spearman Correlation for different word embeddings on Standford rare word dataset
word2vec: (0.2516646845987402)

glove6B 50dimensions (0.26159587465252532, 1782)

glove 100dimensions (0.2136083090117197, 1782)

glove 200dimensions (0.17667532980978479, 1782)

glove 300dimensions (0.15571816294824803, 1782)

glove 42B 300dimensions 

fasttext

In [4]:
def get_spearman_corr(word_vectors):
    rw = pd.read_csv('../dataset/rw/rw.txt', header=None,sep='\t')
    rare_word_pair = []
    for index, row in rw.iterrows():
        rare_word_pair.append((row[0], row[1]))
    corr_list = []
    for word1,word2 in rare_word_pair:
        try:
            cor, p = spearmanr(word_vectors.wv[word1], word_vectors.wv[word2])
            corr_list.append(cor)
        except:
            pass
    return np.mean(corr_list), len(corr_list)
    # w2v 0.2516646845987402

In [ ]:
%time fasttext =KeyedVectors.load_word2vec_format('/home/raja/models/glove300_42B.txt', binary=False)

In [ ]:
get_spearman_corr(fasttext)